* https://www.melon.com/chart/index.htm* 
Melon 100 chart 에서 100곡의 노래 정보를 웹스크래핑 하는 source code를 작성해 주세요.* 
추출할 항목은 노래의 제목, 가수이름, 앨범, 앨범발매일, 장르, 좋아요건수, 가사 입니다.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Melon 100 차트 URL
base_url = 'https://www.melon.com/chart/index.htm'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.melon.com/index.htm'
}

# 페이지 요청 및 파싱
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 곡 정보가 담긴 리스트 가져오기
songs = soup.select('tr[data-song-no]')

# 각 곡의 정보 추출
song_data = []
for idx,song in enumerate(songs,1):
    print(idx)
    song_id = song['data-song-no']
    title = song.select_one('div.ellipsis.rank01 > span > a').text
    artist = song.select_one('div.ellipsis.rank02 > a').text
    album = song.select_one('div.ellipsis.rank03 > a').text

    # 곡의 상세 정보 페이지 요청 및 파싱
    detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    detail_response = requests.get(detail_url, headers=headers)
    detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

    release_date = detail_soup.select_one('dl.list > dd:nth-of-type(2)').text
    genre = detail_soup.select_one('dl.list > dd:nth-of-type(3)').text
    likes = detail_soup.select_one('span.cnt').text.strip('총건수').replace(',', '').strip()

    lyrics = detail_soup.select_one('div.lyric')
    if lyrics:
        lyrics = lyrics.text.strip()
    else:
        lyrics = ''

    song_data.append({
        'title': title,
        'artist': artist,
        'album': album,
        'release_date': release_date,
        'genre': genre,
        'likes': likes,
        'lyrics': lyrics
    })

# Pandas DataFrame으로 변환
df = pd.DataFrame(song_data)

# 결과를 CSV 파일로 저장
df.to_csv('melon_top_100.csv', index=False, encoding='utf-8-sig')

# 결과 출력 (선택사항)
df.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


,title,artist,album,release_date,genre,likes,lyrics
0,Supernova,aespa,Armageddon - The 1st Album,2024.05.13,댄스,총건수\n\t\t\t\t\t\t\t\t\t\t\t0,I’m like some kind of SupernovaWatch outLook a...
1,How Sweet,NewJeans,How Sweet,2024.05.24,댄스,총건수\n\t\t\t\t\t\t\t\t\t\t\t0,All I know is now알게 됐어 나 (I know)그동안 맨날Always ...
2,클락션 (Klaxon),(여자)아이들,I SWAY,2024.07.08,댄스,총건수\n\t\t\t\t\t\t\t\t\t\t\t0,좀 미친 소리 같지만 난 네게 반했어(어어우워 어어우워)자꾸 눈으로 욕 하지마 더 ...
3,Sticky,KISS OF LIFE,Sticky,2024.07.01,댄스,총건수\n\t\t\t\t\t\t\t\t\t\t\t0,How long before we fall in loveHow long before...
4,Small girl (feat. 도경수(D.O.)),이영지,16 Fantasy,2024.06.21,랩/힙합,총건수\n\t\t\t\t\t\t\t\t\t\t\t0,If I got a two small cheeksand a bright pink l...
